# PYTORCH CNN Classifier

In [1]:
DATASET = 'none'
VOCAB_SIZE = 0
TOKENIZER = 'none'
KMER = 0

In [2]:
# Parameters
DATASET = "demo_coding_vs_intergenomic_seqs"
TOKENIZER = "kmer"
KMER = 2


In [3]:
print(DATASET, VOCAB_SIZE, TOKENIZER, KMER)

demo_coding_vs_intergenomic_seqs 0 kmer 2


## Config

In [4]:
import torch
from torch.utils.data import DataLoader

from genomic_benchmarks.dataset_getters.pytorch_datasets import get_dataset
from glp.models import CNN
from glp.tokenizers import get_tokenizer
from glp.tokenizers.utils import build_vocab, coll_factory, check_config, check_seq_lengths

In [5]:
config = {
    "dataset": DATASET,
    "tokenizer": TOKENIZER,
    "dataset_version": 0,
    "epochs": 5,
    "batch_size": 32,
    "use_padding": True,
    "force_download": False,
    "run_on_gpu": True,
    "number_of_classes": 2,
    "embedding_dim": 100,
}
check_config(config)

## Choose the dataset

In [6]:
train_dset = get_dataset(config["dataset"], 'train')

## Tokenizer and vocab

In [7]:
tokenizer = get_tokenizer(config['tokenizer'])
tokenizer.train(train_dset=train_dset, vocab_size=VOCAB_SIZE, kmer=KMER)
vocabulary = build_vocab(train_dset, tokenizer, use_padding=config["use_padding"])

print("vocab len:" ,vocabulary.__len__())
print(vocabulary.get_stoi())

vocab len: 18
{'CG': 16, 'CC': 15, 'GC': 14, 'TC': 13, 'GA': 12, 'TT': 11, 'TG': 10, 'AC': 6, 'CA': 8, 'AG': 3, 'AA': 4, 'TA': 9, '<pad>': 17, 'CT': 7, 'GT': 5, 'AT': 2, 'GG': 1, '<unk>': 0}


## Dataloader and batch preparation

In [8]:
# Run on GPU or CPU
device = 'cuda' if config["run_on_gpu"] and torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

max_tok_len = check_seq_lengths(dataset=train_dset, tokenizer=tokenizer)

# Data Loader
collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = max_tok_len)

train_loader = DataLoader(train_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

Using cpu device


max_tok_len  100


In [9]:
tokenizer(train_dset[1][0])

['CC',
 'AT',
 'CC',
 'GA',
 'GT',
 'GT',
 'CC',
 'TA',
 'GT',
 'GA',
 'AT',
 'CT',
 'GC',
 'AG',
 'CC',
 'CA',
 'GA',
 'TG',
 'GG',
 'CT',
 'CA',
 'CG',
 'TT',
 'TC',
 'CC',
 'TG',
 'TG',
 'TA',
 'CC',
 'CC',
 'AC',
 'AT',
 'GC',
 'CC',
 'AC',
 'TG',
 'AG',
 'CC',
 'CC',
 'GA',
 'AG',
 'TG',
 'GT',
 'TG',
 'CC',
 'CA',
 'GA',
 'CC',
 'CC',
 'TT',
 'AC',
 'CC',
 'AG',
 'AC',
 'CC',
 'CG',
 'GG',
 'CT',
 'GA',
 'AA',
 'GA',
 'AC',
 'CT',
 'TG',
 'TG',
 'TG',
 'GT',
 'TT',
 'AA',
 'TC',
 'TC',
 'AG',
 'AG',
 'CA',
 'GA',
 'GT',
 'CC',
 'TC',
 'TG',
 'AG',
 'CT',
 'GG',
 'AG',
 'AT',
 'GT',
 'TT',
 'GA',
 'TT',
 'GA',
 'GA',
 'GA',
 'CT',
 'GC',
 'CG',
 'GA',
 'GG',
 'GG',
 'CC',
 'TC',
 'CC']

## Model

In [10]:
model = CNN(
    number_of_classes=config["number_of_classes"],
    vocab_size=vocabulary.__len__(),
    embedding_dim=config["embedding_dim"],
    input_len=max_tok_len
).to(device)

## Training

In [11]:
model.train(train_loader, epochs=config["epochs"])

Epoch 0


/Users/katarina/Documents/repositories/GitHub/GLP/src/glp/tokenizers/utils.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(pad(x), dtype=torch.long)


Train metrics: 
 Accuracy: 84.9%, Avg loss: 0.572719 

Epoch 1


Train metrics: 
 Accuracy: 84.6%, Avg loss: 0.568592 

Epoch 2


Train metrics: 
 Accuracy: 86.4%, Avg loss: 0.562357 

Epoch 3


Train metrics: 
 Accuracy: 87.1%, Avg loss: 0.559636 

Epoch 4


Train metrics: 
 Accuracy: 87.8%, Avg loss: 0.557549 



## Testing

In [12]:
test_dset = get_dataset(config["dataset"], 'test')
test_loader = DataLoader(test_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

acc, f1 = model.test(test_loader)
acc, f1

p  12500 ; tp  10191.047959327698 ; fp  1071.4499904791696
recall  0.8152838367462159 ; precision  0.904865688299855
num_batches 782
correct 21655
size 25000
Test metrics: 
 Accuracy: 0.866200, F1 score: 0.857742, Avg loss: 0.563593 



(0.8662, 0.8577421431748532)